In [31]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [74]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

/var/folders/ng/q2m6q2dd6j5_78jgz7wdk2z00000gn/T/ipykernel_82892/4038664820.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Part 1

In [13]:
import numpy as np
from copy import deepcopy
same=2
class PathState:
    def __init__(self,loc,heat, direction, same_dir=0, came_from=None):
        self.loc = loc
        self.heat = heat
        self.direction = direction
        self.same_dir = same_dir
        self.came_from = came_from

    def give_neighbors(self,heat_grid):
        neighs=[]
        nR,nC = len(heat_grid),len(heat_grid[0])
        cur_loc = self.loc
        if self.direction == '>' or self.direction == '<' :
            if self.loc[0] -1 >= 0 :
                neighs.append(PathState((cur_loc[0] -1,cur_loc[1]),
                                         self.heat+heat_grid[cur_loc[0] -1][cur_loc[1]],
                                        '^',
                                        came_from=self
                                       )
                             )
            if self.loc[0] +1 < nR :
                neighs.append(PathState((cur_loc[0] +1,cur_loc[1]),
                                         self.heat+heat_grid[cur_loc[0] +1][cur_loc[1]],
                                        'v',
                                        came_from=self
                                       )
                              
                             )
                
            if self.direction == '>':
                if self.loc[1] +1 < nC and self.same_dir < same :
                    neighs.append(PathState((cur_loc[0],cur_loc[1]+1),
                                             self.heat+heat_grid[cur_loc[0]][cur_loc[1] +1],
                                            '>',
                                            self.same_dir+1,
                                           came_from=self
                                           )
                             )
            else:
                if self.loc[1] -1 >= 0 and self.same_dir < same :
                    neighs.append(PathState((cur_loc[0],cur_loc[1]-1),
                                             self.heat+heat_grid[cur_loc[0]][cur_loc[1] -1],
                                            '<',
                                            self.same_dir+1,
                                           came_from=self
                                           )
                             )
                    
        if self.direction == 'v' or self.direction == '^' :
            if self.loc[1] -1 >= 0 :
                neighs.append(PathState((cur_loc[0],cur_loc[1]-1),
                                         self.heat+heat_grid[cur_loc[0]][cur_loc[1] -1],
                                        '<',
                                        came_from=self
                                       )
                             )
            if self.loc[1] +1 < nC :
                neighs.append(PathState((cur_loc[0],cur_loc[1] +1),
                                         self.heat+heat_grid[cur_loc[0]][cur_loc[1] +1],
                                        '>',
                                        came_from=self
                                       )
                              
                             )
                
            if self.direction == 'v':
                if self.loc[0] +1 < nR and self.same_dir < same :
                    neighs.append(PathState((cur_loc[0]+1,cur_loc[1]),
                                             self.heat+heat_grid[cur_loc[0]+1][cur_loc[1]],
                                            'v',
                                            self.same_dir+1,
                                        came_from=self
                                           )
                             )
            else:
                if self.loc[0] -1 >= 0 and self.same_dir < same :
                    neighs.append(PathState((cur_loc[0]-1,cur_loc[1]),
                                             self.heat+heat_grid[cur_loc[0]-1][cur_loc[1]],
                                            '^',
                                            self.same_dir+1,
                                        came_from=self
                                           )
                             )
        return neighs
                   
    def __repr__(self):
        return f'({self.loc[0]},{self.loc[1]}) with {self.heat} going {self.direction}'
    
    def __str__(self):
        return f'{self.loc[0]}-{self.loc[1]}_{self.direction}_{self.same_dir}'
    
    def __lt__(self,other):
        return self.heat < other.heat
    

In [24]:
import heapq

class PriorityQueue:
    def __init__(self):
        self.elements: list[tuple[float, PathState]] = []
    
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, item: PathState, priority: float):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self) -> PathState:
        return heapq.heappop(self.elements)[1]

In [70]:
from collections import deque

def perform_breadth_first(heat_map):
     # print out what we find
    frontier = PriorityQueue()
    frontier.put( PathState((0,0),0, '>'), priority=0)
    frontier.put( PathState((0,0),0, 'v'), priority=0)
    
    nR,nC = len(heat_map), len(heat_map[0])
    
    reached_min = {}
    
    while not frontier.empty() :
        cur_loc = frontier.get()
        
        if cur_loc.loc == (nR-1,nC-1):
            print(cur_loc.heat)
            return cur_loc
        
        for n_loc in cur_loc.give_neighbors(heat_map):
            seen_key = str(n_loc)
            
            if seen_key not in reached_min.keys() or n_loc.heat < reached_min[seen_key]:
                frontier.put(n_loc, n_loc.heat)
                reached_min[seen_key] = n_loc.heat
                
    return reached_min

In [71]:
def do_part_one(file_path):
    with open(file_path,'r') as f:
        heat_loss = [ [int(i) for i in l.strip()] for l in f.readlines()]
    
    heat = perform_breadth_first(heat_loss)
    nR,nC = len(heat_loss), len(heat_loss[0])
                                          
    blank = [ ['.']*nC for r in range(nR)]
   

    spot = heat
    while spot is not None:
        loc = spot.loc
        blank[loc[0]][loc[1]] = spot.direction
        spot = spot.came_from
    
    
    print('\n'.join([ ''.join(row) for row in blank]))
 #   
    return heat

In [72]:
%%time
tmp = do_part_one('input_data/test_17.txt')

102
v>>..^>>>....
..v>>>..v....
........v>>..
..........v..
..........v>.
...........v.
...........v.
...........v>
............v
............v
...........<v
...........v.
...........v>
CPU times: user 15.2 ms, sys: 2.04 ms, total: 17.3 ms
Wall time: 15.6 ms


In [76]:
%%time
do_part_one('input_data/day_17.txt')

1004
>>...........................................................................................................................................
.v>..........................................................................................................................................
..v>.........................................................................................................................................
...v.........................................................................................................................................
...v.........................................................................................................................................
...v>>.......................................................................................................................................
.....v.......................................................................................................................................
.

(140,140) with 1004 going v

# Part 2

In [83]:
import numpy as np
from copy import deepcopy
class UltimatePathState(PathState):
    def give_neighbors(self,heat_grid):
        neighs=[]
        nR,nC = len(heat_grid),len(heat_grid[0])
        cur_loc = self.loc
        
        change = {'>':  (0,1),
                  '<':  (0,-1),
                  'v':  (1,0),
                  '^':  (-1,0),
                 }
        
        if self.same_dir < 3 :
            next_loc = tuple([i+j for i,j in zip(cur_loc,change[self.direction])])
            if (next_loc[0] > -1 and next_loc[0] < nR 
                and next_loc[1] > -1 and next_loc[1] < nC):
                
                neighs.append(UltimatePathState(next_loc,
                                        self.heat+heat_grid[next_loc[0]][next_loc[1]],
                                        self.direction,
                                        self.same_dir+1,
                                        came_from=self
                                       )
                             )
            return neighs
                
                
            
        
        if self.direction == '>' or self.direction == '<' :
            if self.loc[0] -1 >= 0 :
                neighs.append(UltimatePathState((cur_loc[0] -1,cur_loc[1]),
                                         self.heat+heat_grid[cur_loc[0] -1][cur_loc[1]],
                                        '^',
                                        came_from=self
                                       )
                             )
            if self.loc[0] +1 < nR :
                neighs.append(UltimatePathState((cur_loc[0] +1,cur_loc[1]),
                                         self.heat+heat_grid[cur_loc[0] +1][cur_loc[1]],
                                        'v',
                                        came_from=self
                                       )
                              
                             )
                
        if self.direction == 'v' or self.direction == '^' :
            if self.loc[1] -1 >= 0 :
                neighs.append(UltimatePathState((cur_loc[0],cur_loc[1]-1),
                                         self.heat+heat_grid[cur_loc[0]][cur_loc[1] -1],
                                        '<',
                                        came_from=self
                                       )
                             )
            if self.loc[1] +1 < nC :
                neighs.append(UltimatePathState((cur_loc[0],cur_loc[1] +1),
                                         self.heat+heat_grid[cur_loc[0]][cur_loc[1] +1],
                                        '>',
                                        came_from=self
                                       )
                              
                             )
                
        if self.same_dir < 9:
            next_loc = tuple([i+j for i,j in zip(cur_loc,change[self.direction])])
            
            if (next_loc[0] > -1 and next_loc[0] < nR 
                and next_loc[1] > -1 and next_loc[1] < nC):
                neighs.append(UltimatePathState(next_loc,
                                        self.heat+heat_grid[next_loc[0]][next_loc[1]],
                                        self.direction,
                                        self.same_dir+1,
                                        came_from=self
                                       )
                             )
                
        return neighs


In [84]:
from collections import deque

def ultimate_path(heat_map):
     # print out what we find
    frontier = PriorityQueue()
    frontier.put( UltimatePathState((0,0),0, '>'), priority=0)
    frontier.put( UltimatePathState((0,0),0, 'v'), priority=0)
    
    nR,nC = len(heat_map), len(heat_map[0])
    
    reached_min = {}
    
    while not frontier.empty() :
        cur_loc = frontier.get()
        
        if cur_loc.loc == (nR-1,nC-1):
            print(cur_loc.heat)
            return cur_loc
        
        for n_loc in cur_loc.give_neighbors(heat_map):
            seen_key = str(n_loc)
            
            if seen_key not in reached_min.keys() or n_loc.heat < reached_min[seen_key]:
                frontier.put(n_loc, n_loc.heat)
                reached_min[seen_key] = n_loc.heat
                
    return reached_min

In [85]:
def do_part_two(file_path):
    with open(file_path,'r') as f:
        heat_loss = [ [int(i) for i in l.strip()] for l in f.readlines()]
    
    heat = ultimate_path(heat_loss)
    nR,nC = len(heat_loss), len(heat_loss[0])
                                          
    blank = [ ['.']*nC for r in range(nR)]

    spot = heat
    while spot is not None:
        loc = spot.loc
        blank[loc[0]][loc[1]] = spot.direction
        spot = spot.came_from
    
    
    print('\n'.join([ ''.join(row) for row in blank]))
 #   
    return heat

In [86]:
%%time
do_part_two('input_data/test_17.txt')
           

94
>>>>>>>>>....
........v....
........v....
........v....
........v>>>>
............v
............v
............v
............v
............v
............v
............v
............v
CPU times: user 16.2 ms, sys: 2.29 ms, total: 18.5 ms
Wall time: 16.7 ms


(12,12) with 94 going v

In [87]:
%%time
do_part_two('input_data/day_17.txt')
           

1171
>>>>>>............................................^>>>>>>>>>>.........^>>>>>>>>>>............................................................
.....v.........^>>>>>>>>>>....^>>>>>>>>>>.........^.........v.........^.........v............................................................
.....v.........^.........v....^.........v.........^.........v.........^.........v............................................................
.....v.........^.........v....^.........v.........^.........v.........^.........v............................................................
.....v.........^.........v....^.........v.........^.........v>>>>>>>>>>.........v>>>>>>>>>>..................................................
.....v>>>>>>>>>>.........v>>>>>.........v.........^.......................................v..................................................
........................................v>>>>>>>>>>.......................................v..................................................
.

(140,140) with 1171 going v